In [ ]:
import numpy as np
import cv2
from keypoints import Pipe
import os

action_list = []
video_num_list = []
frame_num_list = []
pose_list = []
face_list = []
left_hand_list = []
right_hand_list = []

# instantiate mediapipe model
model = Pipe()

# set directory for raw data and directory for data post processing
root_raw = './Raw_Data'
root_processed = './Processed_Data'

raw_data_files = os.listdir(root_raw)
actions = ['wave', 'salute', 'kiss', 'idle', 'heart', 'finger']

# set up holistic model
with model.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for file in raw_data_files:
        print(file)
        for action in actions:
            file_path = os.path.join(root_processed, action, file)
            if os.path.isdir(file_path)==False:
                pass
            for video_num in range(model.num_sample_videos):
                print(file, action, video_num)
                file_path = os.path.join(root_processed, action, file, str(video_num))
                if os.path.isdir(file_path)==False:
                    pass
                
                for frame_num in range(model.frames_per_seq):

                    # create the path to the raw video frame
                    frame_path = os.path.join(root_raw, file, action, str(video_num), str(frame_num) + '.npy')
                    
                    
                    # load the numpy frame
                    frame = np.load(frame_path)

                    # make mediapipe detections
                    frame, results = model.pose_detection(frame, holistic)

                    # draw landmarks on frame to be rendered
                    #model.draw_landmarks(frame, results)

                    #cv2.imshow('frame', frame)

                    # extract pose, face, lh, & rh landmarks
                    pose_lm, face_lm, left_hand_lm, right_hand_lm = model.extract_landmarks2(results)
                    
                    
                    
                    action_list.append(action)
                    video_num_list.append(video_num)
                    frame_num_list.append(frame_num)
                    
                    pose_list.append(pose_lm)
                    face_list.append(face_lm)
                    left_hand_list.append(left_hand_lm)
                    right_hand_list.append(right_hand_lm)
                    

In [ ]:

pose_list_df = pd.DataFrame(data=pose_list, columns=np.arange(0, len(pose_list[0])))
face_list_df = pd.DataFrame(data=face_list, columns=np.arange(0, len(face_list[0])))
lh_list_df = pd.DataFrame(data=left_hand_list, columns=np.arange(0, len(left_hand_list[0])))
rh_list_df = pd.DataFrame(data=right_hand_list, columns=np.arange(0, len(right_hand_list[0])))

my_dfs = [pose_list_df, face_list_df, lh_list_df, rh_list_df]
for df in my_dfs:
    df['action'] = action_list
    df['video num'] = video_num_list
    df['frame num'] = frame_num_list
    
pose_list_df.to_csv('pose_landmark_sanity_check.csv')
face_list_df.to_csv('face_landmark_sanity_check.csv')
lh_list_df.to_csv('left_hand_landmark_sanity_check.csv')
rh_list_df.to_csv('right_hand_landmark_sanity_check.csv')